In [18]:
import itertools
from itertools import groupby
import pandas as pd
from pymongo import MongoClient
import json

In [27]:
class DataProcesser(object):
    def __init__(self):
        self.client = MongoClient('localhost', port=27017)
        self.db = self.client.train
        self.train = self.db.Train
        self.station = self.db.Station 

        self.train_df = pd.DataFrame(self.train.find({}, {'_id': 0}))
        self.train_df = self.train_df.drop_duplicates(subset='No', keep='first', inplace=False)

        self.station_df = pd.DataFrame(self.station.find({}, {'_id': 0}))
        self.station_df = self.station_df.drop_duplicates(subset='name', keep='first', inplace=False)

        self.type_dict = {'高速动车组': 'G', '普通动车组': 'D', '城际动车组': 'C', '快速': 'K',
                          '普快': '9', '直达特快': 'Z', '特快': 'T', '其他': 'O'}
                          
    def addTrainsOfStation(self):
        """
        将各个站点的经停列车加入Station表单
        :return: None
        """
        station = self.train_df.key.tolist()
        passby = list(zip(self.train_df.code, self.train_df.key))
        # 添加经过车站的列车
        trainsOfStation = {}
        for (code, names) in passby:
            for name in names:
                if name in trainsOfStation:
                    trainsOfStation[name].append(code)
                elif name != "":
                    trainsOfStation[name] = []
        for (name, codes) in trainsOfStation.items():
            #print(name, codes)              
            query = {"name":name}
            newvalues = { "$set": { "trains": codes } }
            self.station.update_one(query, newvalues)

    def generateMergedTrain(self):
        resLs = []
        # 起止站作为第一特征，以距离作为第二特征量
        documentsDict = {}
        #counter = 0
        for index,document in self.train_df.iterrows():
            start_s = document["start_s"]
            end_s = document["end_s"]
            km = document["km"]
            key1 = f"{start_s}-{end_s}-{km}"
            key2 = f"{end_s}-{start_s}-{km}"
            # counter += 1
            # if counter > 10:
            #     break
            # 起止有问题的不要
            if start_s == "" or end_s == "":
                continue
            # 分正向与反向两种情况讨论
            if key1 in documentsDict:
                documentsDict[key1]["forward"].append(document["code"])
            elif key2 in documentsDict:
                documentsDict[key2]["backward"].append(document["code"])
            else:
                documentsDict[key1] = {"start_s":start_s, "end_s":end_s, "type":document["type"],"distance":[], "time":[], "stations":[], "forward": [document["code"]], "backward":[]}

                prev = [0, 0, 0, 0, 0, 0, "0:00"]
                for item in document["info"]:
                    if item[1] == "": #筛去空站
                        continue
                    documentsDict[key1]["stations"].append(item[1])
                    documentsDict[key1]["distance"].append(int(item[5])-int(prev[5]))
                    prev_t = prev[6].split(':')
                    prev_t = int(prev_t[0])*60+int(prev_t[1])
                    cur_t = item[6].split(':')
                    cur_t = int(cur_t[0])*60+int(cur_t[1])
                    documentsDict[key1]["time"].append(cur_t-prev_t)
                    prev = item
        with open("MergedTrainInfo.json", 'w', encoding ="utf-8") as fw:
            fw.write(json.dumps(list(documentsDict.values()), ensure_ascii=False))
# 实例化数据处理器
processer = DataProcesser()

In [ ]:
# 向数据库加入车站经停列车信息
processer.addTrainsOfStation()

In [28]:
# 将同一线路的列车合并，构造新的列车信息json文件
processer.generateMergedTrain()